In [1]:
# ! pip install mlxtend
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.frequent_patterns import association_rules

sales_fact_1998_ori_df = pd.read_csv("./hw1_data/sales_fact_1998.csv")
sales_fact_dec_1998_ori_df = pd.read_csv("./hw1_data/sales_fact_dec_1998.csv")
product_ori_df = pd.read_csv("./hw1_data/product.csv")
# display(sales_fact_1998_ori_df.info(), sales_fact_dec_1998_ori_df.info(), product_ori_df.info())

time: 244 ms (started: 2022-01-05 02:21:56 +00:00)


In [2]:
transaction_ori_df = pd.concat([sales_fact_1998_ori_df, sales_fact_dec_1998_ori_df]).reset_index()
transaction_df = transaction_ori_df.copy().drop(['promotion_id', 'unit_sales', 'index', 'store_sales', 'store_cost'], axis=1)
transaction_df['product_id'] = transaction_ori_df['product_id'].apply(lambda x:product_ori_df.loc[product_ori_df['product_id']==x, 'product_name'].values[0])
transaction_df = pd.concat([transaction_df, pd.get_dummies(transaction_df['product_id'])], axis=1).drop(['product_id'],axis=1)
transaction_df = transaction_df.drop_duplicates()
transaction_df = transaction_df.groupby(['time_id', 'customer_id', 'store_id']).sum()

time: 33.4 s (started: 2022-01-05 02:21:56 +00:00)


In [3]:
# an_col = [col for col in transaction_df.columns if (set(transaction_df[col].unique().tolist()) != {0,1})]
# duplicate_ind = []
# for ind in transaction_df.loc[(transaction_df==2).any(axis=1),an_col].index:
#     cond = np.logical_and(np.logical_and(transaction_ori_df['time_id']==ind[0], transaction_ori_df['customer_id']==ind[1]), transaction_ori_df['store_id']==ind[2])
#     tmp = transaction_ori_df.loc[cond , 'product_id'].duplicated()
#     duplicate_ind.append(tmp.loc[tmp==1].index.values[0])
    
# duplicate_ind

time: 288 µs (started: 2022-01-05 02:22:30 +00:00)


In [4]:
apriori_freq_itemset = apriori(transaction_df, min_support=0.0001, use_colnames=True, low_memory=True)
apriori_rules = association_rules(apriori_freq_itemset, metric='confidence', min_threshold=0.9)
display(apriori_freq_itemset.sort_values(by = 'support', ascending=False))
display(apriori_rules.sort_values(by = ['lift', 'antecedents'], ascending=False).head(145))
apriori_rules.sort_values(by = ['lift', 'antecedents'], ascending=False).head(145).to_csv('apriori_product.csv')

,support,itemsets
1084,0.004148,(Moms Roasted Chicken)
746,0.004095,(Great English Muffins)
1389,0.003989,(Super Chunky Peanut Butter)
748,0.003963,(Great Pumpernickel Bread)
1117,0.003936,(Nationeel Dried Apples)
...,...,...
9085,0.000106,"(Even Better 2% Milk, Toucan Canned Mixed Fruit)"
9086,0.000106,"(Even Better 2% Milk, Walrus Light Wine)"
9088,0.000106,"(Hermanos Fancy Plums, Even Better Blueberry Y..."
9090,0.000106,"(Imagine Lime Popsicles, Even Better Blueberry..."


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1218,"(Sunset Copper Pot Scrubber, Carlson Large Cur...","(Club 2% Milk, National Small Brown Eggs)",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1217,"(Sunset Copper Pot Scrubber, Club 2% Milk)","(National Small Brown Eggs, Carlson Large Curd...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1220,"(National Small Brown Eggs, Carlson Large Curd...","(Sunset Copper Pot Scrubber, Club 2% Milk)",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1219,"(Club 2% Milk, National Small Brown Eggs)","(Sunset Copper Pot Scrubber, Carlson Large Cur...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1247,"(Club String Cheese, Tri-State Macintosh Apples)","(Horatio Fudge Cookies, Tell Tale Shitake Mush...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
...,...,...,...,...,...,...,...,...,...
1600,"(Akron City Map, Carlson Sharp Cheddar Cheese,...","(Cutting Edge Potato Salad, Pleasant Rice Soup)",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1604,"(Cutting Edge Potato Salad, PigTail Beef TV Di...","(Akron City Map, Carlson Sharp Cheddar Cheese)",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1245,"(Horatio Fudge Cookies, Club String Cheese)","(Tri-State Macintosh Apples, Tell Tale Shitake...",0.000106,0.000132,0.000106,1.0,7570.20,0.000106,inf
1237,"(Great Pumpernickel Bread, Carrington Turkey T...","(High Quality Toilet Paper, Good Merlot Wine)",0.000106,0.000132,0.000106,1.0,7570.20,0.000106,inf


time: 3.56 s (started: 2022-01-05 02:22:30 +00:00)


In [5]:
fp_freq_itemset = fpgrowth(transaction_df, min_support=0.0001, use_colnames=True)
fp_rules = association_rules(fp_freq_itemset, metric='confidence', min_threshold=0.9)
display(fp_freq_itemset.sort_values(by = 'support', ascending=False))
display(fp_rules.sort_values(by = ['lift', 'antecedents'], ascending=False).head(145))
fp_rules.sort_values(by = ['lift', 'antecedents'], ascending=False).head(145).to_csv('fp_product.csv')

,support,itemsets
383,0.004148,(Moms Roasted Chicken)
454,0.004095,(Great English Muffins)
595,0.003989,(Super Chunky Peanut Butter)
1509,0.003963,(Great Pumpernickel Bread)
1032,0.003936,(Carrington Ice Cream)
...,...,...
7334,0.000106,"(Best Choice Dried Apples, Genteel Extra Lean ..."
7335,0.000106,"(Best Choice Dried Apples, Pleasant Canned Tun..."
7336,0.000106,"(Best Choice Dried Apples, Nationeel Mini Donuts)"
7337,0.000106,"(Ebony Broccoli, Best Choice Dried Apples)"


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
848,"(Best Choice Salsa Dip, Quick Extra Lean Hambu...","(Pleasant Fancy Canned Oysters, CDR Hot Chocol...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
914,"(Fast Corn Chips, High Quality Silver Cleaner)","(Dollar Monthly Home Magazine, Fabulous Orange...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
913,"(Dollar Monthly Home Magazine, Fabulous Orange...","(Fast Corn Chips, High Quality Silver Cleaner)",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
853,"(Best Choice Salsa Dip, Quick Extra Lean Hambu...","(Pleasant Fancy Canned Oysters, Imagine Low Fa...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1320,"(Best Choice Salsa Dip, Quick Extra Lean Hambu...","(Pleasant Fancy Canned Oysters, CDR Hot Chocol...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
...,...,...,...,...,...,...,...,...,...
1475,"(Shady Lake Spaghetti, Tell Tale Cauliflower)","(Horatio Cheese Dip, Just Right Chicken Ramen ...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1460,"(Shady Lake Spaghetti, Tell Tale Beets)","(Tell Tale Limes, Horatio Frosted Donuts)",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
859,"(Super Grape Preserves, Big Time Blueberry Waf...","(Booker Large Curd Cottage Cheese, Discover Ra...",0.000106,0.000132,0.000106,1.0,7570.20,0.000106,inf
849,"(Best Choice Salsa Dip, Quick Extra Lean Hambu...","(Pleasant Fancy Canned Oysters, Imagine Low Fa...",0.000106,0.000132,0.000106,1.0,7570.20,0.000106,inf


time: 1.33 s (started: 2022-01-05 02:22:33 +00:00)


In [6]:
display(apriori_rules, fp_rules)
display(apriori_rules.sort_values(by = ['lift', 'antecedents'], ascending=False).head(145))
display(fp_rules.sort_values(by = ['lift', 'antecedents'], ascending=False).head(145))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Hermanos Firm Tofu, Booker Low Fat Sour Cream)",(ADJ Rosy Sunglasses),0.000106,0.003699,0.000106,1.0,270.364286,0.000105,inf
1,"(Hermanos Firm Tofu, Even Better Large Curd Co...",(ADJ Rosy Sunglasses),0.000106,0.003699,0.000106,1.0,270.364286,0.000105,inf
2,"(Akron City Map, Carlson Sharp Cheddar Cheese)",(Cutting Edge Potato Salad),0.000106,0.003012,0.000106,1.0,332.026316,0.000105,inf
3,"(Akron City Map, Carlson Sharp Cheddar Cheese)",(PigTail Beef TV Dinner),0.000106,0.002985,0.000106,1.0,334.964602,0.000105,inf
4,"(Akron City Map, PigTail Beef TV Dinner)",(Carlson Sharp Cheddar Cheese),0.000106,0.003408,0.000106,1.0,293.418605,0.000105,inf
...,...,...,...,...,...,...,...,...,...
2094,"(Horatio No Salt Popcorn, Cormorant Scented To...","(Just Right Canned Yams, Faux Products HCL Nas...",0.000106,0.000159,0.000106,1.0,6308.500000,0.000106,inf
2095,"(Just Right Canned Yams, Hilltop 200 MG Acetom...","(Faux Products HCL Nasal Spray, Horatio No Sal...",0.000106,0.000132,0.000106,1.0,7570.200000,0.000106,inf
2096,"(Just Right Canned Yams, Cormorant Scented Toi...","(Hilltop 200 MG Acetominifen, Faux Products HC...",0.000106,0.000132,0.000106,1.0,7570.200000,0.000106,inf
2097,"(Hilltop 200 MG Acetominifen, Cormorant Scente...","(Just Right Canned Yams, Faux Products HCL Nas...",0.000106,0.000132,0.000106,1.0,7570.200000,0.000106,inf


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(High Top Oranges, BBB Best Apple Butter)",(Bravo Fancy Canned Anchovies),0.000106,0.003382,0.000106,1.0,295.710938,0.000105,inf
1,"(Sunset AAA-Size Batteries, Moms Turkey Hot Dogs)",(Jeffers Oatmeal),0.000106,0.003170,0.000106,1.0,315.425000,0.000105,inf
2,"(Tri-State Honey Dew, Fabulous Cranberry Juice)",(Faux Products Whitening Toothpast),0.000106,0.003144,0.000106,1.0,318.075630,0.000105,inf
3,"(Tri-State Honey Dew, Faux Products Whitening ...",(Fabulous Cranberry Juice),0.000106,0.003514,0.000106,1.0,284.593985,0.000105,inf
4,"(Best Choice Potato Chips, Faux Products White...",(Super Strawberry Preserves),0.000106,0.003170,0.000106,1.0,315.425000,0.000105,inf
...,...,...,...,...,...,...,...,...,...
2094,"(PigTail Ice Cream Sandwich, Imagine Frozen Ch...",(Excellent Diet Soda),0.000106,0.003223,0.000106,1.0,310.254098,0.000105,inf
2095,"(Excellent Diet Soda, Kiwi Lox)","(PigTail Ice Cream Sandwich, Imagine Frozen Ch...",0.000106,0.000106,0.000106,1.0,9462.750000,0.000106,inf
2096,"(PigTail Ice Cream Sandwich, Imagine Frozen Ch...","(Excellent Diet Soda, Kiwi Lox)",0.000106,0.000106,0.000106,1.0,9462.750000,0.000106,inf
2097,"(PigTail Ice Cream Sandwich, Kiwi Lox)","(Excellent Diet Soda, Imagine Frozen Chicken T...",0.000106,0.000159,0.000106,1.0,6308.500000,0.000106,inf


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1218,"(Sunset Copper Pot Scrubber, Carlson Large Cur...","(Club 2% Milk, National Small Brown Eggs)",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1217,"(Sunset Copper Pot Scrubber, Club 2% Milk)","(National Small Brown Eggs, Carlson Large Curd...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1220,"(National Small Brown Eggs, Carlson Large Curd...","(Sunset Copper Pot Scrubber, Club 2% Milk)",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1219,"(Club 2% Milk, National Small Brown Eggs)","(Sunset Copper Pot Scrubber, Carlson Large Cur...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1247,"(Club String Cheese, Tri-State Macintosh Apples)","(Horatio Fudge Cookies, Tell Tale Shitake Mush...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
...,...,...,...,...,...,...,...,...,...
1600,"(Akron City Map, Carlson Sharp Cheddar Cheese,...","(Cutting Edge Potato Salad, Pleasant Rice Soup)",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1604,"(Cutting Edge Potato Salad, PigTail Beef TV Di...","(Akron City Map, Carlson Sharp Cheddar Cheese)",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1245,"(Horatio Fudge Cookies, Club String Cheese)","(Tri-State Macintosh Apples, Tell Tale Shitake...",0.000106,0.000132,0.000106,1.0,7570.20,0.000106,inf
1237,"(Great Pumpernickel Bread, Carrington Turkey T...","(High Quality Toilet Paper, Good Merlot Wine)",0.000106,0.000132,0.000106,1.0,7570.20,0.000106,inf


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
848,"(Best Choice Salsa Dip, Quick Extra Lean Hambu...","(Pleasant Fancy Canned Oysters, CDR Hot Chocol...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
914,"(Fast Corn Chips, High Quality Silver Cleaner)","(Dollar Monthly Home Magazine, Fabulous Orange...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
913,"(Dollar Monthly Home Magazine, Fabulous Orange...","(Fast Corn Chips, High Quality Silver Cleaner)",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
853,"(Best Choice Salsa Dip, Quick Extra Lean Hambu...","(Pleasant Fancy Canned Oysters, Imagine Low Fa...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1320,"(Best Choice Salsa Dip, Quick Extra Lean Hambu...","(Pleasant Fancy Canned Oysters, CDR Hot Chocol...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
...,...,...,...,...,...,...,...,...,...
1475,"(Shady Lake Spaghetti, Tell Tale Cauliflower)","(Horatio Cheese Dip, Just Right Chicken Ramen ...",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
1460,"(Shady Lake Spaghetti, Tell Tale Beets)","(Tell Tale Limes, Horatio Frosted Donuts)",0.000106,0.000106,0.000106,1.0,9462.75,0.000106,inf
859,"(Super Grape Preserves, Big Time Blueberry Waf...","(Booker Large Curd Cottage Cheese, Discover Ra...",0.000106,0.000132,0.000106,1.0,7570.20,0.000106,inf
849,"(Best Choice Salsa Dip, Quick Extra Lean Hambu...","(Pleasant Fancy Canned Oysters, Imagine Low Fa...",0.000106,0.000132,0.000106,1.0,7570.20,0.000106,inf


time: 43.3 ms (started: 2022-01-05 02:22:35 +00:00)
